In [1]:
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd

In [111]:
timestart = '2025-04-05 00:00:00'
timeend = '2025-04-07 00:00:00'

input_file = '../data/active_dispatch_4_5.csv'
output_file = '../data/active_dispatch_ticks.csv'

In [113]:
tick = 1
current_tick_time = dt.strptime(timestart, "%Y-%m-%d %H:%M:%S")
end_tick_time = dt.strptime(timeend, "%Y-%m-%d %H:%M:%S")

In [115]:
processed_ticks = pd.DataFrame(columns = ['tick','code','date','hr','time','address','ert_mins','lat','lon'])  # (Estimated Resolution Time: ert) calculated from last_updated

In [117]:
# next steps:
# open the current csv
# loop through each tick, performing a query on if the current tick is between the start (date & time as datetime) and estimated resolved (as start + ert_mins)
     # whatever the query results are, add them to the dataframe.
     # if blank, ignore?  this dataset will only have a blank in the beginning.

In [119]:
ad = pd.read_csv(input_file)

In [121]:
how_many_minutes = 5 # how long each tick will be

In [123]:
ad['date_and_time'] = ad['date'] + ' ' + ad['time']
ad['date_and_time'] = pd.to_datetime(ad['date_and_time'])

In [125]:
for index, row in ad.iterrows() :
    ad.loc[index,'estimated_resolved_time'] = row['date_and_time'] + timedelta(minutes = row['ert_mins'])

In [127]:
while current_tick_time <= end_tick_time :

    current_ticks = ad[(current_tick_time <= ad['estimated_resolved_time']) & (current_tick_time >= ad['date_and_time'])].reset_index(drop = True)
    current_ticks['tick'] = tick
    current_ticks = current_ticks[['tick','code','date','hr','time','address','ert_mins','lat','lon']]

    processed_ticks = pd.concat([processed_ticks, current_ticks])
    
    tick += 1
    current_tick_time += timedelta(minutes = how_many_minutes)

C:\Users\Garrett\AppData\Local\Temp\ipykernel_5104\78851439.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  processed_ticks = pd.concat([processed_ticks, current_ticks])


In [129]:
processed_ticks

,tick,code,date,hr,time,address,ert_mins,lat,lon
0,3,64P,4/5/2025,0,0:06:56,"3056 DICKERSON PIKE, NASHVILLE, TN",15,36.229127,-86.759745
0,4,64P,4/5/2025,0,0:06:56,"3056 DICKERSON PIKE, NASHVILLE, TN",15,36.229127,-86.759745
0,5,64P,4/5/2025,0,0:06:56,"3056 DICKERSON PIKE, NASHVILLE, TN",15,36.229127,-86.759745
1,5,52P,4/5/2025,0,0:19:45,"1410 DOUGLAS AVE, NASHVILLE, TN",11,36.191982,-86.740490
0,6,52P,4/5/2025,0,0:19:45,"1410 DOUGLAS AVE, NASHVILLE, TN",11,36.191982,-86.740490
...,...,...,...,...,...,...,...,...,...
9,577,70A,4/6/2025,23,23:20:16,"924 CANTRELL AVE, NASHVILLE, TN",41,NaN,NaN
10,577,57P,4/6/2025,23,23:23:18,"703 BATTLE RD, NASHVILLE, TN",89,NaN,NaN
11,577,71A,4/6/2025,23,23:38:52,"928 RIVERGATE PKWY, NASHVILLE, TN",22,NaN,NaN
12,577,83P,4/6/2025,23,23:41:55,"305 STARBOARD CT, NASHVILLE, TN",19,NaN,NaN


In [ ]:
# Next to try for geocoding...
# process with arcgis, as much as possible
# census next
# google webscrape next
# another alternative is Nominatim

# Plan A: ArcGIS

In [47]:
from geopy.geocoders import ArcGIS

In [49]:
def geocode(address_or_intersection) :
    address_or_intersection = address_or_intersection.replace("/", "&") # For an intersection, it needs to be & not /
    
    n = ArcGIS().geocode(address_or_intersection)
    return n[0], n[1]

In [53]:
address, coordinates = geocode('BELLEVUE RD / HICKS RD, NASHVILLE, TN')
print(address)
print(coordinates)
print(coordinates[0])

Bellevue Rd & Hicks Rd, Nashville, Tennessee, 37221
(36.062545979425, -86.923692965379)
36.062545979425


In [55]:
address, coordinates = geocode('608 W DUE WEST AVE, NASHVILLE, TN')
print(address)
print(coordinates)
print(coordinates[0])

608 W Due West Ave, Madison, Tennessee, 37115
(36.249367699273, -86.735595671686)
36.249367699273


In [57]:
address, coordinates = geocode('COMMERCE ST & 6TH AVE N, NASHVILLE, TN')
print(address)
print(coordinates)
print(coordinates[0])

Commerce St & 6th Ave N, Nashville, Tennessee, 37203
(36.161345986725, -86.78057004082)
36.161345986725


# Plan B: Census
### Seems like this should work well for intersections, I just need to code it still, ha ha

In [43]:
# Code it!
# https://geocoding.geo.census.gov/geocoder/locations/onelineaddress?address=GREEN%20LN%20%26%20TISDALL%20DR,%20NASHVILLE,%20TN&benchmark=Public_AR_Current&format=json
# json format
# Use %26 as an & (and instead of /)

In [45]:
# Looks like it should be something like...
# result: addressMatches: 0: coordinates: x & y

# Plan C: Google web scrape
### Seems to work well on addresses and on intersections (with &, not /)
### If I go this route, definately space it out with sleep... I don't want to get Google mad at my scraping!

In [18]:
import requests
import re

In [19]:
address = '608 W DUE WEST AVE, NASHVILLE, TN'

In [22]:
res = requests.get('https://www.google.com/maps/place/'+address)

In [24]:
# search for gps in the format "https://www.google.com/maps/preview/place/Hillsboro+Pike+%26+Otter+Creek+Rd,+Forest+Hills,+TN+37215/@36.0680982,-86.8441147,
# Nashville is always going to be 36, -86 lat/lon, so I'm looking for that - sign specifically...
test = re.search(r'"https://www.google.com/maps/preview/place/(.+)/@(\d+\.\d+),(-\d+\.\d+),', res.text)
print(test.group(1))
print(test.group(2))
print(test.group(3))

608+W+Due+West+Ave,+Madison,+TN+37115
36.2493773
-86.7355026


In [26]:
intersection1 = "COMMERCE ST & 6TH AVE N, NASHVILLE, TN"
intersection2 = "COMMERCE ST / 6TH AVE N, NASHVILLE, TN"

In [28]:
res = requests.get('https://www.google.com/maps/place/'+intersection1)

In [30]:
test = re.search(r'"https://www.google.com/maps/preview/place/(.+)/@(\d+\.\d+),(-\d+\.\d+),', res.text)
print(test.group(1))
print(test.group(2))
print(test.group(3))

Commerce+St+%26+6th+Ave+N,+Nashville,+TN+37203
36.161315
-86.7805606


In [32]:
res = requests.get('https://www.google.com/maps/place/'+intersection2)

In [34]:
test = re.search(r'"https://www.google.com/maps/preview/place/(.+)/@(\d+\.\d+),(-\d+\.\d+),', res.text)
print(test.group(1))
print(test.group(2))
print(test.group(3))

Commerce+St,+New+York,+NY+10014
40.7315538
-74.0049385


# Plan D: Nominatim
### Seems to work well for addresses, but not intersections.
### If I have to go this route, maybe search more documentation... is there a way to encode intersections?

In [1]:
from geopy.geocoders import Nominatim

# Instantiate a new Nominatim client
app = Nominatim(user_agent="Nashville Police Dispatch and Weather Project")

location = app.geocode("3056 DICKERSON PIKE, NASHVILLE, TN").raw

# Print raw data
print(location)

{'place_id': 370499868, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 48729772, 'lat': '36.2278548', 'lon': '-86.7596889', 'class': 'place', 'type': 'house', 'place_rank': 30, 'importance': 9.99999999995449e-06, 'addresstype': 'place', 'name': '', 'display_name': '3056, Dickerson Pike, Nashville, Davidson County, Middle Tennessee, Tennessee, 37207, United States', 'boundingbox': ['36.2278048', '36.2279048', '-86.7597389', '-86.7596389']}


In [14]:
intersection = "COMMERCE ST & 6TH AVE N, NASHVILLE, TN"
intersection = "COMMERCE ST / 6TH AVE N, NASHVILLE, TN"

#location = app.geocode(intersection).raw

# print(location)

In [ ]:
# This one seems to be good for addresses, but not intersections with & or /, produces an error.